# __Experiments__

In [1]:
import math
import pandas as pd
import numpy as np
import nltk
import time
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, defaultdict
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.grid_search import GridSearchCV
import scipy.stats

In [2]:
data_path = '../data/clean_data_full.csv'
df = pd.read_csv(data_path)
# df.info()

In [3]:
## Num rows
print "num observations:\t", len(df)

num observations:	52778


# Feature Engineering

In [4]:
def unigram_phi(review):
    return Counter(review.split())
def bigram_phi(review):
    return Counter(nltk.bigrams(review.split()))
def unigram_bigram_phi(review):
    return unigram_phi(review) + bigram_phi(review)
def trigram_phi(review):
    return Counter(nltk.trigrams(review.split()))

# Learners

In [5]:
class Baseline:
    def __init__(self):
        self.avg_respsponse = 0
    def fit(self, X, y):
        self.avg_response = np.mean(y)
    def predict(self, X):
        rows, _ = X.shape
        return np.repeat(self.avg_response, rows)

def fit_baseline(X, y):
    """
    Naive baseline return mean training prediction
    """
    mod = Baseline()
    mod.fit(X, y)
    return mod

In [6]:
def fit_linear_regression(X, y):    
    """
    Linear Regression
    """
    mod = LinearRegression(fit_intercept=True, n_jobs = -1)
    mod.fit(X, y)
    return mod

In [7]:
def fit_lasso(X, y, alpha = 0.1, max_iter = 1000):
    lasso = Lasso(alpha = alpha, max_iter = max_iter)
    mod = lasso.fit(X, y)
    return mod

In [8]:
def fit_ridge(X, y, alpha = 0.1, max_iter = 1000):
    lasso = Lasso(alpha = alpha, max_iter = max_iter)
    mod = lasso.fit(X, y)
    return mod

In [73]:
def fit_gbm_regression(X, y,
                       n_estimators = 100,
                       learning_rate = 0.1,
                       max_depth = 1,
                       random_state = 0,
                       loss = "ls"):
    """
    Gradient Boosting Method Regression
    """
    gbm = GradientBoostingRegressor(n_estimators = n_estimators,
                                    learning_rate = learning_rate,
                                    max_depth = max_depth,
                                    random_state = random_state,
                                    loss = "ls")
    mod = gbm.fit(X.toarray(), y)
    return mod

In [141]:
def fit_rf_regression(X, y,
                      n_estimators = 100,
                      max_depth = math.sqrt,
                      max_features = 'auto',
                      n_jobs = -1):
    """
    Gradient Boosting Method Regression
    """
    ## Restrict depth with fn if None is not passed
    ## expect somthing like sqrt() to be passed here
    max_depth = max_depth(X.shape[1]) if max_depth != None else max_depth
    rf = RandomForestRegressor(n_estimators = n_estimators,
                               max_depth = max_depth,
                               max_features = max_features,
                               n_jobs = n_jobs)
    mod = rf.fit(X.toarray(), y)
    return mod

In [11]:
def fit_adaBoost_regression(X, y,
                            n_estimators = 50,
                            learning_rate = 1,
                            loss = "linear"):
    """
    Gradient Boosting Method Regression
    """
    adaBoost = AdaBoostRegressor(n_estimators = n_estimators,
                           learning_rate = learning_rate,
                           loss = loss)
    mod = adaBoost.fit(X.toarray(), y)
    return mod

In [12]:
def build_data_set(data, vectorizer = None, aspect_str = "OVERALL"):
    """
    Aspect ratings cols:
        (7)  review_palate_score      -- score_normalizer = 1
        (8)  review_taste_score       -- score_normalizer = 2
        (11) review_aroma_score       -- score_normalizer = 2
        (14) review_avg_score         -- score_normalizer = 1
        (18) review_overall_score     -- score_normalizer = 4
        (20) review_appearance_score  -- score_normalizer = 1
        
    predict_col :: column for aspect we're predicting, current default is column 18 (OVERALL)
    
    """
    
    ## RateBeer scrape data locations
    ## ------------------------------
    REVIEW_BLOB = 24
    ASPECTS = {
        "PALATE"     : [7, 1],
        "TASTE"      : [8, 2],
        "AROMA"      : [11, 2],
        "AVERAGE"    : [14, 1],
        "OVERALL"    : [18, 4],
        "APPEARANCE" : [20, 1]
    }
    assert aspect_str in ASPECTS
        
    aspect_column = ASPECTS[aspect_str][0]      ## Get aspect rating column
    aspect_normalizer = ASPECTS[aspect_str][1]  ## Get aspect normalizer
    labels = []                                 ## Ratings
    feat_dicts = []                             ## Features
    raw_examples = []                           ## Review strings
    data_values = data.values                   ## Data from pandas df
    for row in data.values:
        review, score = row[REVIEW_BLOB], row[aspect_column]
        score = float(score) / aspect_normalizer

        ## Safety check
        if not isinstance(review, basestring):
            print 'weird review:\t', review
            
#         feat_dicts.append(phi(review))
        labels.append(score)
        raw_examples.append(review)
        
    # In training, we want a new vectorizer:
    if vectorizer == None:
        vectorizer = DictVectorizer(sparse=True)
        feat_matrix = vectorizer.fit_transform(feat_dicts)
    # In assessment, we featurize using the existing vectorizer:
    else:
        feat_matrix = vectorizer.fit_transform(raw_examples)

    return {'X'            : feat_matrix, 
            'y'            : labels, 
            'vectorizer'   : vectorizer, 
            'raw_examples' : raw_examples,
            'feature_names' : vectorizer.get_feature_names()}

In [180]:
def experiment(data,
               model = fit_baseline,
               phi = None,
               assess_data = None,
               train_size = 0.9,
               score_metric = mean_squared_error,
               verbose = False):
    ## Check that we're running a model
    assert(model != None)
    
    ## Timing
    start_time = time.time()
    
    ## Build data set
    X_train = data['X'] 
    y_train = data['y']
    vectorizer = data['vectorizer']
    feature_names = data['feature_names']

    ## Test-train split
    if assess_data == None:
        X_train, X_assess, y_train, y_assess = train_test_split(
                X_train, y_train, train_size = train_size)
    ## Only use for test-set
    else:
        assess = build_data_set(assess_data, phi, vectorizer = vectorizer)
        X_assess, y_assess = assess['X'], assess['y']
        

    ## Model data
    mod = model(X_train, y_train)
    predictions = mod.predict(X_assess.toarray())
    
    ## Timing
    run_time = time.time() - start_time

    if verbose:
        print "\tExperiment information"
        print '\t======================='
        print "\ttrained:\t", model.__name__
        print "\tnum training observations:\t", X_train.shape[0]
        print "\tnum training features:\t", X_train.shape[1]
        print "\tscore_metric:\t", score_metric.__name__
        print "\trun time: ", run_time
#         if model.__name__ == 'fit_linear_regression' or\
#             model.__name__ == 'fit_lasso' or\
#             model.__name__ == 'fit_ridge':
# #             print feature_names[1:10]
# #             print mod.coef_[1]
        print "y_assess[1:10]:\t", y_assess[1:10]
        print "predictions[1:10]:\t", predictions[1:10]
    
    ## Return results dict - will write to csv
    res = {
        "score"        : score_metric(y_assess, predictions),
        "score_name"   : score_metric.__name__,
        "predictions"  : predictions,
        "y_assess"     : y_assess
    }
    return res
    

In [14]:
def train_reader(train_path = data_path, n = 1000):
    df = pd.read_csv(data_path)
    return df.sample(n)

def dev_reader(test_path, n = 100):
    df = pd.read_csv(data_path)
    return df.sample(n)

# Vectorizer's encode phi...

In [27]:
## bigram_vectorizer
## -----------------
## normalization:
##  * lower
##  * remove stop words
##  * min word length == 2
##
bigram_vectorizer = CountVectorizer(analyzer='word', stop_words = 'english', ngram_range=(2, 2), min_df = 3, max_features = 5000)

In [28]:
## unigram_vectorizer
## -----------------
## normalization:
##  * lower
##  * remove stop words
##  * min word length == 2
##
unigram_vectorizer = CountVectorizer(analyzer='word', stop_words = 'english', ngram_range=(1, 1), min_df = 3, max_features = 5000)

In [29]:
## trigram_vectorizer
## ------------------
## normalization:
##  * lower
##  * remove stop words
##  * min word length == 2
##
trigram_vectorizer = CountVectorizer(analyzer='word', stop_words = 'english', ngram_range=(3, 3), min_df = 3, max_features = 5000)

In [18]:
def set_vectorizer(n_gram = "unigram",
                   min_df = 3,
                   sample_size = 10000,
                   stop_words = 'english',
                   features_prop = 0.25,
                   max_features = 2000):

    max_features = int(sample_size * features_prop) if max_features == None else max_features
    if n_gram == "trigram":
        ngram_range = (3,3)
    elif n_gram == "bigram":
        ngram_range = (2,2)
    else: # default to unigram
        ngram_range = (1,1)
    
    return CountVectorizer(analyzer='word',
                           ngram_range = ngram_range,
                           min_df = min_df,
                           max_features = max_features,
                           stop_words = stop_words)

# Data

In [181]:
# def build_data_set(data, phi, vectorizer = None, aspect_str = "OVERALL"):

n_samples = 40000
vectorizer = set_vectorizer(n_gram = "unigram", sample_size = n_samples, features_prop = 0.05, max_features = None)
train_d = train_reader(data_path, n = n_samples)
built_data_set = build_data_set(train_d, vectorizer = vectorizer, aspect_str = 'TASTE')

# Prelim runs

### Baseline

In [117]:
baseline_results = experiment(data = built_data_set,
           model = fit_baseline,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print baseline_results['score_name']
print baseline_results['score']

	Experiment information
	trained:	fit_baseline
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  0.0844750404358
y_assess[1:10]:	[4.0, 3.5, 3.5, 3.5, 2.5, 3.0, 5.0, 4.5, 1.0]
predictions[1:10]:	[ 3.40823611  3.40823611  3.40823611  3.40823611  3.40823611  3.40823611
  3.40823611  3.40823611  3.40823611]
-----------------------------
mean_squared_error
0.954698024498


### Linear regresssion

In [132]:
## Note: running time ~ 1 minute @ 40k observations
## Note on number of features :: lookes like LM with n = 40,000 obs prefers 1,200 features
lm_results = experiment(data = built_data_set,
           model = fit_linear_regression,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print lm_results['score_name']
print lm_results['score']

	Experiment information
	trained:	fit_linear_regression
	num training observations:	9000
	num training features:	500
	score_metric:	mean_squared_error
	run time:  0.0764510631561
[u'12', u'12oz', u'2015', u'2016', u'absolutely', u'abv', u'acidic', u'actually', u'aftertaste']
0.0975684074743
y_assess[1:10]:	[4.0, 5.0, 3.0, 3.5, 4.0, 3.0, 1.5, 2.0, 3.5]
predictions[1:10]:	[ 3.65662724  3.49659209  3.06649112  3.5858353   3.965771    3.56893541
  2.83095575  3.58264942  2.94071635]
-----------------------------
mean_squared_error
0.703292786185


### Regularized Regression

In [119]:
# n_samples = 20000
# vectorizer = set_vectorizer(n_gram = "unigram", sample_size = n_samples, features_prop = 1, max_features = None)
# train_d = train_reader(data_path, n = n_samples)
# built_data_set = build_data_set(train_d, vectorizer = vectorizer, aspect_str = 'TASTE')

#### Ridge regression

In [124]:
## Note: running time few seconds @ 40k observations
ridge_results = experiment(data = built_data_set,
           model = fit_ridge,
           verbose = True, 
           score_metric = mean_squared_error)
print "-----------------------------"
print ridge_results['score_name']
print ridge_results['score']

	Experiment information
	trained:	fit_ridge
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  0.227540016174
[u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'100']
0.0
y_assess[1:10]:	[3.5, 4.5, 3.5, 3.5, 4.0, 2.5, 4.0, 3.5, 4.0]
predictions[1:10]:	[ 3.40380556  3.40380556  3.40380556  3.40380556  3.40380556  3.40380556
  3.40380556  3.40380556  3.40380556]
-----------------------------
mean_squared_error
0.913640537809


#### Lasso

In [125]:
## Note: running time few seconds @ 40k observations
lasso_results = experiment(data = built_data_set,
           model = fit_lasso,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print lasso_results['score_name']
print lasso_results['score']

	Experiment information
	trained:	fit_lasso
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  0.223304986954
[u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'100']
0.0
y_assess[1:10]:	[4.0, 5.0, 4.0, 4.0, 3.5, 3.5, 4.0, 3.0, 3.5]
predictions[1:10]:	[ 3.40316667  3.40316667  3.40316667  3.40316667  3.40316667  3.40316667
  3.40316667  3.40316667  3.40316667]
-----------------------------
mean_squared_error
0.887641277778


### Random Forest

In [142]:
## Note: run time ~3min with 10K obs and 10 trees
##                ~8min with 40K obs and 0.05 * 40K (2000)features and 10 trees
##                ~
rf_results = experiment(data = built_data_set,
           model = fit_rf_regression,
           verbose = True,
           score_metric = mean_squared_error)
print "-----------------------------"
print rf_results['score_name']
print rf_results['score']

KeyboardInterrupt: 

### GBM regression

In [74]:
# def fit_gbm_regression(X, y,
#                        n_estimators = 10,
#                        learning_rate = 0.1,
#                        max_depth = 1,
#                        random_state = 0,
#                        loss = "ls"):

# Note: running time is currently ver long
experiment(data = built_data_set,
           model = fit_gbm_regression,
           verbose = True,
           score_metrics = [mean_squared_error, r2_score])

	Experiment information
	trained:	fit_gbm_regression
	num training observations:	36000
	num training features:	1200
	score_metrics:	['mean_squared_error', 'r2_score']
	run time:  117.637772083
y_assess[1:10]:	[4.5, 4.0, 4.0, 4.5, 4.0, 3.0, 4.5, 5.0, 2.0]
predictions[1:10]:	[ 3.6650699   3.34262176  3.47430726  3.47430726  3.59508346  3.09653888
  3.67842892  3.55909983  3.39723603]


[0.7754653792740176, 0.15726864684880737]

### Ada Boost regression

In [ ]:
# Note: running time is currently ver long
experiment(data = built_data_set,
           model = fit_adaBoost_regression,
           verbose = True,
           score_metric = mean_squared_error)

## Parameter tuning

In [187]:
def tune_learner_params(X, y, basemod, cv, param_grid, scoring = None): 
    """
    Description
        
    """    
    # Find the best model within param_grid:
    crossvalidator = GridSearchCV(basemod, param_grid, cv = cv, scoring=scoring, n_jobs = -1)
    crossvalidator.fit(X, y)
    # Report some information:
    print("Best params", crossvalidator.best_params_)
    print("Best score: %0.03f" % crossvalidator.best_score_)

    return crossvalidator.best_estimator_

In [183]:
def fit_lasso_cv(X, y):
    """
    
    """    
    basemod = Lasso()
    cv = 5
    param_grid = {
        'alpha': [0.01, 0.05, 0.1, 0.2, 0.5], 
        'max_iter': [2, 10, 25, 50, 500]
                 }    
    return tune_learner_params(X, y, basemod, cv, param_grid)

In [184]:
lasso_cv_res = experiment(data = built_data_set,
                          model = fit_lasso_cv,
                          verbose = True,
                          score_metric = mean_squared_error)

('Best params', {'alpha': 0.01, 'max_iter': 2})
Best score: 0.124
	Experiment information
	trained:	fit_lasso_cv
	num training observations:	36000
	num training features:	2000
	score_metric:	mean_squared_error
	run time:  32.2567131519
y_assess[1:10]:	[4.0, 3.5, 4.5, 5.0, 4.0, 1.5, 2.0, 1.5, 5.0]
predictions[1:10]:	[ 3.70219408  3.36351796  3.29294267  3.45679579  3.58465853  3.01585312
  3.29294267  3.36194337  3.45460068]


In [178]:
print lasso_cv_res['score']

0.839102674598


In [186]:
def fit_rf_cv(X, y):
    """
    
    """    
    basemod = RandomForestRegressor()
    cv = 5
    param_grid = {
        'n_estimators': [1, 10, 100], 
        'max_features': [1, 'sqrt', 'log2', 'auto']
                 }    
    return tune_learner_params(X, y, basemod, cv, param_grid)

In [ ]:
rf_cv_res = experiment(data = built_data_set,
                          model = fit_rf_cv,
                          verbose = True,
                          score_metric = mean_squared_error)

## Tune num features

In [36]:
def tune_max_features(n_observations = 10000,
                      n_gram = "unigram",
                      tuning_values = [0.05, 0.10, 0.2, 0.3],
                      model = fit_rf_regression,
                      score_metric = mean_squared_error,
                      verbose = False):

    start_time = time.time()
    
    res = defaultdict(float)
    for tuning_value in tuning_values:
        if verbose:
            print "Currently tuning:\t", model.__name__
            print "n_gram:\t", n_gram
            print "tuning_value:\t", tuning_value
            
        vectorizer = set_vectorizer(n_gram = n_gram,
                                sample_size = n_observations,
                                features_prop = tuning_value)
        train_d = train_reader(data_path, n = n_samples)
        built_data_set = build_data_set(train_d, vectorizer = vectorizer, aspect_str = 'TASTE')
        res[str(tuning_value)] = experiment(data = built_data_set,
                                            model = model,
                                            verbose = False,
                                            score_metrics = [mean_squared_error])

    run_time = time.time() - start_time    
    print "runtime for " + str(n_observations) + ":\t", run_time
    return res

In [37]:
## Records
# n = 5000; runtime = 235sec; best 0.3 (0.69)
# n = 2000; runtime = 60sec
# n = 10000, alpha = 0.2, mse = 0.63

tune_max_features(n_observations = 10000, n_gram = "unigram", verbose = True)

Currently tuning:	fit_rf_regression
n_gram:	unigram
tuning_value:	0.05
Currently tuning:	fit_rf_regression
n_gram:	unigram
tuning_value:	0.1
Currently tuning:	fit_rf_regression
n_gram:	unigram
tuning_value:	0.2


KeyboardInterrupt: 

# Between aspects - lasso

In [ ]:
ngram = 'bigram'
ASPECTS = ['OVERALL', 'TASTE', 'AROMA', 'PALATE', 'APPEARANCE']
for aspect in ASPECTS:
    n_samples = 40000
    vectorizer = set_vectorizer(n_gram = ngram, sample_size = n_samples)
    train_d = train_reader(data_path, n = n_samples)
    built_data_set = build_data_set(train_d, vectorizer = vectorizer, aspect_str = aspect)

    print "\n-----------------"
    print "ngram:\t", ngram
    print "curr aspect:\t", aspect
    mse = experiment(data = built_data_set,
           model = fit_lasso,
           verbose = True)
    print "mse:\t", mse

# Multiple runs

In [128]:
learners = [fit_linear_regression, fit_lasso, fit_gbm_regression]
vectorizers = [unigram_vectorizer, bigram_vectorizer]
vectorizer_names = ["unigram", "bigram"]

In [132]:
for learner in learners:
    print "\n============================="
    print "Fitting:\t", learner.__name__
    for vectorizer, name in zip(vectorizers, vectorizer_names):
        print "-------"
        print "vectorizer:\t", name
        mse = experiment(train_d,
                         learner,
                         unigram_phi,
                         vectorizer = vectorizer,
                         verbose = True)
        print "mse:\t", mse


Fitting:	fit_linear_regression
-------
vectorizer:	unigram


KeyboardInterrupt: 

# Model testing

In [283]:
def run_model(d_sample, model, phi, n_samples = 100):
    res = [experiment(data, model, phi) for _ in range(n_samples)]
    return res

In [284]:
glm_res = run_model(train, fit_linear_regression, trigram_phi)
gbm_res = run_model(train, fit_gbm_regression, trigram_phi)
lasso_res = run_model(train, fit_lasso, trigram_phi)

In [285]:
print "glm:", np.mean(glm_res), np.var(glm_res)
print "gbm:", np.mean(gbm_res), np.var(gbm_res)
print "lasso", np.mean(lasso_res), np.var(lasso_res)

glm: 5.09104430619 1.65291139452
gbm: 5.4193393923 2.1477179012
lasso 5.5710284521 1.65857267327


In [288]:
print scipy.stats.wilcoxon(glm_res, gbm_res)[1]
print scipy.stats.wilcoxon(glm_res, lasso_res)[1]
print scipy.stats.wilcoxon(gbm_res, lasso_res)[1]

0.0933661297989
0.0212407456292
0.5025560931
